# Responsi 2

# Nama : Arvinanto Bahtiar
# NIM : 2211110014

GitHub link: https://github.com/suxeno/chatbot-mentalhealth

In [12]:
import pandas as pd
import numpy as np
import random
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import re

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import textwrap

In [3]:
nltk.download("wordnet")
nltk.download("punkt")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
df = pd.read_csv("dataset_mentalhealth.csv")

In [6]:
df

,Question_ID,Questions,Jawaban
0,1590140,Apa yang dimaksud dengan penyakit mental?,Penyakit mental adalah kondisi kesehatan yang ...
1,2110618,Siapa yang terpengaruh oleh penyakit mental?,Diperkirakan bahwa penyakit mental mempengaruh...
2,6361820,Apa penyebab penyakit mental?,Diperkirakan bahwa penyakit mental mempengaruh...
3,9434130,Apa sajakah tanda-tanda peringatan penyakit me...,Gejala gangguan kesehatan mental bervariasi te...
4,7657263,Apakah penderita penyakit jiwa bisa sembuh?,"Ketika penyembuhan dari penyakit mental, ident..."
...,...,...,...
93,4373204,Bagaimana saya tahu kalau saya minum terlalu b...,Menyortir jika Anda minum terlalu banyak bisa ...
94,7807643,"Jika ganja berbahaya, mengapa kita melegalkannya?","Asap ganja, misalnya, mengandung racun penyeba..."
95,4352464,Bagaimana cara meyakinkan anak saya untuk tida...,Anda tidak bisa. Tetapi Anda dapat mempengaruh...
96,6521784,Apa status hukum (dan bukti) minyak CBD?,Cannabidiol atau CBD adalah komponen ganja yan...


## Question text cleaning

In [7]:
# Inisialisasi lemmatizer
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Membersihkan teks dari karakter khusus dan mengubahnya menjadi huruf kecil
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Lemmatisasi kata-kata
    tokens = nltk.word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token, pos='v') for token in tokens]
    return " ".join(lemmatized_tokens)

# Menerapkan preprocessing pada kolom teks
df["Questions"] = df["Questions"].apply(preprocess_text)

In [8]:
df.head(10)

,Question_ID,Questions,Jawaban
0,1590140,apa yang dimaksud dengan penyakit mental,Penyakit mental adalah kondisi kesehatan yang ...
1,2110618,siapa yang terpengaruh oleh penyakit mental,Diperkirakan bahwa penyakit mental mempengaruh...
2,6361820,apa penyebab penyakit mental,Diperkirakan bahwa penyakit mental mempengaruh...
3,9434130,apa sajakah tandatanda peringatan penyakit mental,Gejala gangguan kesehatan mental bervariasi te...
4,7657263,apakah penderita penyakit jiwa bisa sembuh,"Ketika penyembuhan dari penyakit mental, ident..."
5,1619387,apa yang harus saya lakukan jika saya mengenal...,Meskipun situs web ini tidak dapat menggantika...
6,1030153,bagaimana saya bisa menemukan ahli kesehatan m...,Merasa nyaman dengan profesional yang Anda ata...
7,8022026,pilihan pengobatan apa yang tersedia,Sama seperti ada berbagai jenis obat untuk pen...
8,1155199,jika saya terlibat dalam pengobatan apa yang p...,Karena memulai perawatan adalah langkah besar ...
9,7760466,apa perbedaan antara profesional kesehatan mental,Ada banyak jenis profesional kesehatan mental....


## TFxIDF approach for transforming text

In [9]:
# Inisialisasi TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()
# Melakukan fit transform pada kolom pertanyaan untuk mendapatkan representasi TF-IDF
tfidf_matrix = tfidf_vectorizer.fit_transform(df["Questions"])

## Chatbot with cosine similarity matching method

In [10]:
# Fungsi untuk mendapatkan pertanyaan yang paling mirip berdasarkan cosine similarity
def get_most_similar_answer(input_text, df):
    input_vector = tfidf_vectorizer.transform([input_text])
    cosine_similarities = cosine_similarity(input_vector, tfidf_matrix)
    most_similar_index = np.argmax(cosine_similarities)
    most_similar_answer = df.loc[most_similar_index, "Jawaban"]
    return most_similar_answer

In [14]:
def suggest_questions(df):
    questions = df["Questions"].tolist()
    random.shuffle(questions)  # Acak urutan pertanyaan
    return questions[:5]  # Ambil 5 pertanyaan pertama

def chatbot(df):
    print("Selamat datang di Chatbot Kesehatan Mental!")

    while True:
        print("\nSuggested Questions:")
        for i, suggestion in enumerate(suggest_questions(df), 1):
            print(f"{i}. {suggestion}")
        print()
        user_input = input("Masukkan pertanyaan Anda (atau ketik 'exit' untuk keluar): ")
        if user_input.lower() == 'exit':
            print("\nTerima kasih telah menggunakan chatbot.")
            break
        else:
            user_input = preprocess_text(user_input)
            response = get_most_similar_answer(user_input, df)
            # Memformat respons agar rapih
            formatted_response = textwrap.fill(response, width=80)
            print("\nJawaban dari chatbot:")
            print(formatted_response)
            print("\n")

# Contoh penggunaan chatbot
chatbot(df)

Selamat datang di Chatbot Kesehatan Mental!

Suggested Questions:
1. apakah ada obat untuk masalah kesehatan mental
2. apa yang harus saya lakukan jika saya mengenal seseorang yang tampaknya memiliki gejala gangguan jiwa
3. apa yang harus saya ketahui sebelum memulai pengobatan baru
4. bisakah anda mencegah masalah kesehatan mental
5. di mana saya dapat menemukan informasi lebih lanjut tentang depresi

Masukkan pertanyaan Anda (atau ketik 'exit' untuk keluar): bisakah saya mencegah masalah mental

Jawaban dari chatbot:
Kita semua dapat menderita tantangan kesehatan mental, tetapi mengembangkan
kesejahteraan, ketahanan, dan mencari bantuan kita sejak dini dapat membantu
mencegah tantangan menjadi serius.



Suggested Questions:
1. bagaimana saya bisa menjaga kesehatan fisik saya
2. apa perbedaan antara kecemasan dan gangguan kecemasan
3. apa perbedaan antara psikiater dan psikolog terdaftar
4. di mana saya dapat menemukan layanan kesehatan mental anak dan remaja
5. apa itu gangguan sikl

## Interpretasi

Secara overview keseluruhan proses pengerjaan chatbot ini hanya cukup menggunakan TfxIDF sebagai method untuk meng vektorisasi kata-kata pada setiap questions template yang nanti nya akan di match dengan inputan text dari user lalu akan diberikan output derdasarkan dataset yang ada sesuai atau mendekati dengan pertanyaan yang user berikan menggunakan *metrics cosine similarity*. Algoritma chatbot ini cukup mudah diimplementasikan dan sanngat ringan serta efficient karena tidak menggunakan algoritma machine learning atau deep learning sama sekali. Namun tidak dapat dipungkiri chatbot ini masih memiliki kekurangan, salah satunya yang sangat *noticeable* adalah ketika ada 2 data question yang miirp secara vektor pada dtaset sehingga ketika user memberikan input respon dari chatbot terkadang misslead ke dataset lain yang berbeda konteks hanya karena vektor input user sedikit mirip dengan data tersebut.